# Basic Geographical Symbology

In [ ]:
import os
import pandas as pd
import geopandas as gpd
%matplotlib notebook
from shapely.geometry import Point, LineString, Polygon
from fiona.crs import from_epsg
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

As always lets go ahead import our go to libraries and set some settings.

Today is all about symbology. We are going to _symbolize_ each of the typs discussed in our previous lecture. First lets load in some data so that way we have something to work with. 

In [ ]:
os.chdir('data')

Alright we are going to do something a little more advanced to keep everyone from having to run soooo many execution boxes. Functions! If you remember from early on in the semester we talked about these.  They are essentially a series of commands that you can tell python to do within by defining them not individually, but together. Everything we are doing here is what we did last week so don't get to nervous its just bunched together.

In [ ]:
def get_tabular():
    
    food_data = pd.read_csv('food_all.csv', encoding='latin-1')
    
    keep_cols = ['ccode', 'country', 'leader', 'year', 'elected', 'age', 'male',
       'militarycareer', 'government',
       'prev_conflict', 'pt_suc', 'pt_attempt', 'precip', 'couprisk',
       'democracy', 'autocracy', 'day', 'date', 'GEO',
       '_ISO3N_', 'M49 Code_x', 'percap cals']
    
    food_data = food_data[keep_cols]
    
    pts_data = pd.read_csv('PTS-2019.csv', encoding='latin-1')
    
    pts_cols = ['Year', 'COW_Code_N', 'PTS_A', 'PTS_H', 'PTS_S']
    
    pts_data = pts_data[pts_cols]
    
    left_merge = pd.merge(food_data, pts_data, left_on=['year', 'ccode'], right_on=['Year','COW_Code_N'], how='left')
    
    left_merge[left_merge.duplicated(['year', 'ccode'], keep=False)]
    
    left_merge = left_merge.drop_duplicates(subset=['year', 'ccode'], keep='last')
    
    df = left_merge
    
    return df

Now to run our function the way we have set it up, we just call it and set our data frame equal to it.

In [ ]:
food_data = get_tabular()

Lets check and make sure!

In [ ]:
food_data.head()

Perfect! Now lets do a function to get our world shapefile and narrow it down to the most recent version.

In [ ]:
def get_world():
    
    world = gpd.read_file('worldmap/cshapes.shp') 
    
    world = world[world.COWEYEAR == 2016]
    
    return world

In [ ]:
world = get_world()

In [ ]:
world.plot()

Now lets get a little trickier. Today lets say our question is all about the countries we had worked with in Africa last time namely; Central African Republic, DRC, and Uganda. In addition to all of the country level statstics we want two additional aspects (both which we went over last time).

1. We want a cross-sectional data frame that has those countries with a sum of conflict deaths.
2. An event level file to show each individual event as a point.

In [ ]:
def make_two_dfs(food_data, world):
    
    food_data = food_data[food_data.year == 2014] ##snapshot of 2014 for our food, regime, and state charicteristics data
    
    world = world.merge(food_data, left_on='COWCODE', right_on='ccode', how='left')
    
    countrys = ['Uganda', 'Central African Republic', 'Congo, DRC']
    
    central_af = world[world.CNTRY_NAME.isin(countrys)]
    
    # now lets get the two things we want. 
    
    ged_2014 = pd.read_csv('ged_2014.csv')
    
    country_sums = ged_2014.groupby('country_id')['best'].sum().reset_index() ## LEts get the sum of deaths by country
    
    central_af = central_af.merge(country_sums, left_on='COWCODE', right_on='country_id', how='left') ## And here we satisfy our first requirement
    
    codes = [482, 490, 500]
    
    ged_car = ged_2014[ged_2014.country_id.isin(codes)]
    
    ged_geometry = [Point(xy) for xy in zip(ged_car.longitude, ged_car.latitude)] ## Lets make our geometry 
    
    ged_gdf = gpd.GeoDataFrame(ged_car, geometry = ged_geometry) ## Finally lets get number 2
    
    return central_af, ged_gdf

In [ ]:
world2, ged_gdf = make_two_dfs(food_data, world)

Lets check and make sure we got what we want.

In [ ]:
world2.head()

In [ ]:
ged_gdf.tail()

Look at all those functions put to good use!  Now instead of line after line of executing files, we just have a few and everything is a little smoother. We are well on our way to becoming more _Pythonic_ coders!

Now that we have our base data from last time all loaded up, lets get to what we want to do today, symbolizing:
- Points
- Lines
- Polygons

## Points

Lets start with points. First lets do a quick points plots like last time.

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray');
ged_gdf.plot(ax=ax, color='blue');
ax.set_aspect('equal')

Now we got all the points plotted onto our map. Great! How about we vary the size some now.

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray');
ged_gdf.plot(ax=ax, color='blue', markersize=ged_gdf['best']);
ax.set_aspect('equal')

So is this proportional or graduated???

If you guessed __

![correct](https://media.giphy.com/media/PS7d4tm1Hq6Sk/giphy.gif)

Lets do the other one then. First we should check the distribution of the variable.

In [ ]:
ged_gdf.best.describe()

So lets take this continuous variable and chop it up into bins or classes.

In [ ]:
def bin_it(ged_gdf):
    
    bins = [-1, 1.1, 3.1, 8.1, 500]
    labels = [2,5,10,20]
    ged_gdf['best_bin'] = pd.cut(ged_gdf['best'], bins,labels=labels)
    ged_gdf['best_bin'] = ged_gdf['best_bin'].astype(int)
    
    return ged_gdf

In [ ]:
ged_gdf2 = bin_it(ged_gdf)

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray');
#scatter = ged_gdf2.plot(ax=ax, color='green', markersize=ged_gdf2['best_bin']);
scatter = ax.scatter(ged_gdf2['longitude'], ged_gdf2['latitude'], color='green', s=ged_gdf2['best_bin'])
ax.set_aspect('equal')

So we now have some proportional and graduated dots on the map.  How about we vary some other aspects of the dots.

### Color

In addition to varying the color, we can also add some other things to our map to make it a little more appealing. Edgecolor, alphas (opacity), and a background color. 

For color lets find a variable that has a few classes that would be important to see the distinction between, how about type of violence.

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray', edgecolor='black');
scatter = ax.scatter(ged_gdf2['longitude'], ged_gdf2['latitude'], c=ged_gdf2.type_of_violence, cmap='PiYG', s=ged_gdf2['best_bin'],
                    alpha=0.7)
ax.set_aspect('equal')
ax.set_facecolor('steelblue')
cbar = fig.colorbar(scatter, ticks=[1, 2, 3])
cbar.ax.set_yticklabels(['State', 'Non-State', 'One-Sided'])


Go ahead and vary some of the colors! Why not choose a new color map. https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

Some color maps will not work well for this one.  Remember your variable types (and what the range is within that variable) is important when thinking about color choices.

Also check out the color wheels and complimentary colors here: https://www.canva.com/colors/color-wheel/

For our next excercise on markers: https://matplotlib.org/1.4.1/api/markers_api.html


### Symbols

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray', edgecolor='black');
state = ged_gdf2.loc[ged_gdf2.type_of_violence ==1]
non = ged_gdf2.loc[ged_gdf2.type_of_violence ==2]
one = ged_gdf2.loc[ged_gdf2.type_of_violence ==3]

scatter1 = ax.scatter(state['longitude'], state['latitude'], color='blue', marker='x', s=ged_gdf2['best'],
                    alpha=0.7)
scatter2 = ax.scatter(non['longitude'], non['latitude'], color='green', marker='v', s=ged_gdf2['best'],
                    alpha=0.7)
scatter3 = ax.scatter(one['longitude'], one['latitude'], color='yellow', marker='d', s=ged_gdf2['best'],
                    alpha=0.7)
ax.set_aspect('equal')
ax.set_facecolor('steelblue')

Lets try again with some colors that are actually appealing.  Use the color wheel to pick your own colors!

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray', edgecolor='black');
state = ged_gdf2.loc[ged_gdf2.type_of_violence ==1]
non = ged_gdf2.loc[ged_gdf2.type_of_violence ==2]
one = ged_gdf2.loc[ged_gdf2.type_of_violence ==3]

scatter1 = ax.scatter(state['longitude'], state['latitude'], color='#25DAC3', marker='x', s=ged_gdf2['best'],
                    alpha=0.7)
scatter2 = ax.scatter(non['longitude'], non['latitude'], color='#C325DA', marker='v', s=ged_gdf2['best'],
                    alpha=0.7)
scatter3 = ax.scatter(one['longitude'], one['latitude'], color='#DAC325', marker='d', s=ged_gdf2['best'],
                    alpha=0.7)
ax.set_aspect('equal')
ax.set_facecolor('steelblue')

![alright](https://media.giphy.com/media/g5zvwUa9720pO/giphy.gif)

We are making progress!!

Lets move on to changing some lines now.

## Lines

First we can load in some data on rivers.

In [ ]:
rivers = gpd.read_file('central_africa/rivers.shp') 

In [ ]:
rivers.plot(linewidth=rivers.StrokeWeig)

In [ ]:
rivers.head()

Luckily we have something built into this data that gives us a clean value for the river sizes.  Let's add this to our last plot!

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray', edgecolor='black');
state = ged_gdf2.loc[ged_gdf2.type_of_violence ==1]
non = ged_gdf2.loc[ged_gdf2.type_of_violence ==2]
one = ged_gdf2.loc[ged_gdf2.type_of_violence ==3]

scatter1 = ax.scatter(state['longitude'], state['latitude'], color='#25DAC3', marker='x', s=ged_gdf2['best'],
                    alpha=0.7)
scatter2 = ax.scatter(non['longitude'], non['latitude'], color='#C325DA', marker='v', s=ged_gdf2['best'],
                    alpha=0.7)
scatter3 = ax.scatter(one['longitude'], one['latitude'], color='#DAC325', marker='d', s=ged_gdf2['best'],
                    alpha=0.7)
rivers.plot(ax=ax, linewidth=rivers.StrokeWeig)
ax.set_aspect('equal')
ax.set_facecolor('steelblue')

Sweet! Lines can be used just like points. You can vary shape, color and size. Lets do one more with the shape. 

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, facecolor='gray', edgecolor='black');
state = ged_gdf2.loc[ged_gdf2.type_of_violence ==1]
non = ged_gdf2.loc[ged_gdf2.type_of_violence ==2]
one = ged_gdf2.loc[ged_gdf2.type_of_violence ==3]

scatter1 = ax.scatter(state['longitude'], state['latitude'], color='#25DAC3', marker='x', s=ged_gdf2['best'],
                    alpha=0.7)
scatter2 = ax.scatter(non['longitude'], non['latitude'], color='#C325DA', marker='v', s=ged_gdf2['best'],
                    alpha=0.7)
scatter3 = ax.scatter(one['longitude'], one['latitude'], color='#DAC325', marker='d', s=ged_gdf2['best'],
                    alpha=0.7)

rivers.plot(ax=ax, linewidth=rivers.StrokeWeig, linestyle='-.')
ax.set_aspect('equal')
ax.set_facecolor('steelblue')

Zoom in to the map and check it out. You can see that the rivers are actually patterned differently than just a solid line. https://matplotlib.org/3.1.0/gallery/lines_bars_and_markers/linestyles.html


On to the main event, Polygons!

## Polygons

Let's go ahead and turn to choropleth maps now.  Very basically you can vary the color within a cross-section of polygons based on some variable. So what would be a good variable in this data to vary. How about we try a few and use different cmaps.

In [ ]:
world2.head()
#world2.precip.describe()

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, column='precip', cmap='PiYG',facecolor='gray', edgecolor='black', legend=True);
ax.set_aspect('equal')
ax.set_facecolor('steelblue')
plt.title("Precipitation Levels")


Hmmm that one isn't all that pleasing. 

![alright](https://media.giphy.com/media/FYCxLHCIGaioo/giphy.gif)


How about we try one that is sequential.

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, column='precip', cmap='Wistia',facecolor='gray', edgecolor='black', legend=True);
ax.set_aspect('equal')
ax.set_facecolor('steelblue')
plt.title("Precipitation Levels")


Its better but maybe we want to flip it. Something you can do is reverse a color scheme but just using the simple _r

In [ ]:
fig, ax = plt.subplots()
world2.plot(ax=ax, column='precip', cmap='Wistia_r',facecolor='gray', edgecolor='black', legend=True);
ax.set_aspect('equal')
ax.set_facecolor('steelblue')
plt.title("Precipitation Levels")


Exercise: Why don't you all try and make some other choropleth maps.  Use additional columns, colors, and even add some of the points and lines we have done so far!

# BONUS 

Try to load up the ACLED data.  Then narrow down to only protests and riots and plot that.